In [1]:
#단일 입력 PromptTemplate: 하나의 변수로 구성된 프롬프트 템플릿 개념과 예시 (예: 제품 설명 문구 생성)
#다중 입력 PromptTemplate: 둘 이상의 변수를 사용하는 템플릿 예시 (예: 제목과 키워드로 요약문 생성)
#ChatPromptTemplate와 역할 기반 프롬프트: 시스템/사용자 역할별 프롬프트 구성과 ChatPromptTemplate.from_messages() 사용법
#PartialPromptTemplate 활용: 프롬프트 일부를 미리 고정하는 부분 포맷팅 개념과 실용 예 (예: 시스템 메시지는 고정하고 사용자 입력만 변경)
#프롬프트 출력 및 체인 실행: 완성된 프롬프트를 확인하고 LLM과 연결하여 실행하는 방법 (LangChain Expression Language 기반 prompt | llm 파이프라인 사용)
#프롬프트 작성 팁: 프롬프트 템플릿 작성 시 주의할 사항과 실무적인 모범 사례

# 단일입력 PromptTemplate

In [2]:
from langchain.prompts import PromptTemplate

In [4]:
import langchain
langchain.__version__

'0.3.25'

In [11]:
template_str = (
    "카피라이터로서 아래 제품의 홍보 문구 50자 이내로 작성. \n"
    "제품명: {product_name}"
)

prompt = PromptTemplate.from_template(template_str)

input_str= input("제품명: ")
formatted_prompt = prompt.format(product_name="input_str")
print(formatted_prompt)

카피라이터로서 아래 제품의 홍보 문구 50자 이내로 작성. 
제품명: input_str


In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

chain = prompt | llm

result = chain.invoke({"product_name": input_str})
print(result.content)

포터블 게임의 혁신, Nintendo 3DS로 생생한 3D 경험을 즐기세요!


# 다중입력 PromptTemplate

In [17]:
multi_template_str = (
    "뉴스 기사 제목과 키워드를 보고 요약문 작성. \n"
    "제목 : {title}\n"
    "키워드 : {keywords}\n"
)

summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keywords"])

sample_title = "인공지능 기술의 발전과 미래"
sample_keywords = "머신러닝, 딥러닝, 산업, 혁신, 사회"
formatted_summary_prompt = summary_prompt.format(title = sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

뉴스 기사 제목과 키워드를 보고 요약문 작성. 
제목 : 인공지능 기술의 발전과 미래
키워드 : 머신러닝, 딥러닝, 산업, 혁신, 사회



In [21]:
result_summary = (summary_prompt | llm).invoke({
    "title": sample_title,
    "keywords": sample_keywords
})
print(result_summary.content)

인공지능 기술의 발전은 머신러닝과 딥러닝의 발전을 바탕으로 산업 전반에 혁신을 가져오고 있으며, 이러한 변화는 사회 전반에 걸쳐 큰 영향을 미치고 있습니다.


# ChatPromptTemplate
역할기반 프롬프트

In [22]:
from langchain.prompts import ChatPromptTemplate

In [23]:
sys_message = (
    "사용자의 프로그래밍 질문에 전문가로써 답변"
)
user_message = " {question}" #사용자질문 -> 실행시 입력값으로

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_message),
    ("user", user_message)
])

sample_question = "파이썬의 리스트 컴프리헨션과 map의 차이?"

message = chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='사용자의 프로그래밍 질문에 전문가로써 답변', additional_kwargs={}, response_metadata={}), HumanMessage(content=' 파이썬의 리스트 컴프리헨션과 map의 차이?', additional_kwargs={}, response_metadata={})]


In [24]:
result = (chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

파이썬에서 리스트 컴프리헨션(List Comprehension)과 `map()` 함수는 모두 기존 시퀀스(리스트, 튜플 등)를 기반으로 새 리스트를 생성하는 방법이지만, 몇 가지 차이점이 있습니다.

1. 문법과 가독성
   - 리스트 컴프리헨션은 더 직관적이고 간결한 문법을 제공합니다.
     ```python
     # 리스트 컴프리헨션
     squares = [x**2 for x in range(10)]
     ```
   - `map()`은 함수와 시퀀스를 인자로 받으며, 익명 함수 또는 기존 함수를 사용할 수 있습니다.
     ```python
     # map() 사용
     squares = list(map(lambda x: x**2, range(10)))
     ```

2. 함수 사용
   - 리스트 컴프리헨션은 조건문이나 복잡한 표현식을 포함할 수 있어 유연성이 높습니다.
   - `map()`은 주로 기존 함수를 적용하는 데 적합하며, 람다 함수를 함께 사용하면 유사한 기능을 구현할 수 있습니다.

3. 성능
   - 일반적으로 리스트 컴프리헨션이 `map()`보다 약간 빠른 경우가 많습니다. 하지만 이는 상황에 따라 다를 수 있으며, 명확성과 가독성을 우선시하는 것이 좋습니다.

4. 가독성과 사용 용도
   - 간단한 변환이나 필터링에는 리스트 컴프리헨션이 더 직관적입니다.
   - 이미 정의된 함수를 여러 번 적용하거나, 함수형 프로그래밍 스타일을 선호한다면 `map()`이 적합합니다.

요약하면:
| 특징 | 리스트 컴프리헨션 | `map()` |
|---|---|---|
| 문법 | `[expression for item in iterable if condition]` | `map(function, iterable, ...)` |
| 가독성 | 높음 | 중간 |
| 유연성 | 높음 (복잡한 조건, 표현식 가능) | 제한적 (함수 적용에 적합) |
| 성능 | 대체로 빠름 | 비슷하거나 약간 느림 |

**추천:** 

# PartialPromptTemplate

In [25]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [27]:
role_system_template = "{role}로써 자세히 답변"


system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt = base_chat_prompt.partial(role= "전문 투자인")


In [29]:
sample_question = "2025년 5월 시장 상황 기준, 삼성전자 주식 전망은?"
message = (partial_chat_prompt | llm).invoke({"question", sample_question})
print(message)

content='2025년 5월 기준 삼성전자 주식 전망에 대해 전문가의 관점에서 분석해 드리겠습니다. 다만, 주식 시장은 다양한 변수에 영향을 받기 때문에 확정적인 예측은 어렵다는 점을 유념하시기 바랍니다.\n\n1. 글로벌 반도체 시장 동향\n- 삼성전자는 글로벌 반도체 시장의 선도 기업으로, D램과 낸드플래시 시장에서 강력한 위치를 유지하고 있습니다.\n- 2025년까지 반도체 수요는 AI, 5G, 자율주행차, 데이터 센터 등 신기술 확산에 힘입어 지속 성장할 것으로 기대됩니다.\n- 그러나 공급 과잉이나 가격 변동성, 글로벌 공급망 이슈는 리스크 요인입니다.\n\n2. 모바일 및 소비자 가전 부문\n- 삼성전자는 스마트폰, 가전제품 등에서 강력한 브랜드 파워를 보유하고 있으며, 신기술 도입과 프리미엄 제품 확대를 통해 수익성을 높이고 있습니다.\n- 2025년에는 폴더블 스마트폰, IoT 가전 등 신시장 개척이 기대됩니다.\n\n3. 기술 혁신 및 투자 전략\n- 삼성전자는 AI, 3nm 공정 등 첨단 기술 개발에 적극 투자하고 있으며, 이는 경쟁 우위 확보에 기여할 전망입니다.\n- 친환경 및 지속가능경영 전략도 기업 이미지와 장기 성장에 긍정적 영향을 미칠 수 있습니다.\n\n4. 글로벌 경제 및 정책 환경\n- 미국, 중국 등 주요 국가의 무역 정책, 규제, 환율 변동이 삼성전자 실적에 영향을 미칠 수 있습니다.\n- 글로벌 경기 침체 우려가 커질 경우, 반도체 및 IT 수요 둔화 가능성도 고려해야 합니다.\n\n5. 주가 전망\n- 단기적으로는 글로벌 시장 변동성, 환율, 금리 정책 등에 따라 변동성이 클 수 있습니다.\n- 장기적으로는 반도체 및 IT 산업의 성장, 삼성전자의 기술력, 시장 점유율 확대 등을 감안할 때 긍정적인 전망도 존재합니다.\n\n요약하면, 2025년 5월 기준 삼성전자 주식은 글로벌 기술 수요 증가와 기업의 혁신 전략이 긍정적 영향을 미칠 것으로 기대되나, 시장 변동성, 지정학적 리스크 등도 함께 고려해야 합니다. 투자 